In [4]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import os
import json

pd.set_option('display.max_columns', None)
output_path = "./output/population/"

# Loading events to separate tables
* agents: people
* vehicles:
    * cars
    * subway
    * buses
    * trams
    * trains

In [5]:
events = pd.DataFrame()

dtypes = {
    "time": np.float64,
    "type": str,
    "driverId": str,
    "vehicleId": str,
    "transitLineId": str,
    "transitRouteId": str,
    "departureId": str,
    "person": str,
    "link": str,
    "legMode": 'category', #category
    "vehicle": str,
    "networkMode": str, #category
    "relativePosition": np.float64,
    "facility": str,
    "delay": np.float64,
    "x": np.float64,
    "y": np.float64,
    "actType": str,
}

for csv in os.listdir(output_path):
    df = pd.read_csv(output_path+csv, dtype=dtypes)
    events =events.append(df, ignore_index=True)

C:\Users\Anna\.pyenv\pyenv-win\versions\3.9.6\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (19,21,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Anna\.pyenv\pyenv-win\versions\3.9.6\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (16,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Anna\.pyenv\pyenv-win\versions\3.9.6\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Anna\.pyenv\pyenv-win\versions\3.9.6\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code

In [6]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10639855 entries, 0 to 10639854
Data columns (total 25 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   Unnamed: 0                int64  
 1   time                      float64
 2   type                      object 
 3   driverId                  object 
 4   vehicleId                 object 
 5   transitLineId             object 
 6   transitRouteId            object 
 7   departureId               object 
 8   person                    object 
 9   link                      object 
 10  legMode                   object 
 11  vehicle                   object 
 12  networkMode               object 
 13  relativePosition          float64
 14  facility                  object 
 15  delay                     float64
 16  x                         float64
 17  y                         float64
 18  actType                   object 
 19  computationalRoutingMode  object 
 20  distance              

In [47]:
events.legMode.value_counts()
#events.facility.value_counts()

car     76244
walk    42488
pt      22621
Name: legMode, dtype: int64

In [48]:
events.vehicle.value_counts()

veh_25966_bus    1262
veh_25967_bus    1244
veh_25968_bus    1244
veh_26036_bus    1212
veh_26031_bus    1193
                 ... 
veh_16892_bus       7
veh_3066_bus        7
veh_22642_bus       7
veh_17184_bus       7
veh_17138_bus       7
Name: vehicle, Length: 33089, dtype: int64

In [49]:
np.where(events.person == "8")

(array([ 226725,  226726,  226727,  226728,  226729,  986546,  986547,
         986548, 5239512, 5239513, 7411157, 7411158, 7411159, 7411160,
        7411161, 7411162, 8059863, 8059864, 8144600, 8144601, 8144602,
        8144603, 8144604, 8144605, 8147963, 8185403, 8185404, 8185405,
        8185406, 8185407, 8203670, 8203671, 8203672, 8203673, 8203674,
        8203675, 8252485, 8348315, 8348316, 8348317, 8348318, 8348319,
        8459971, 8607820, 8607821, 8607822], dtype=int64),)

In [10]:
person = events.loc[np.where(events.person == 1.0)]
#display(person.head(10))
#person.iloc[np.where(person.legMode == "walk")]

In [11]:
#events.describe()

## Loading people to separate file

In [12]:
agents = pd.DataFrame()
agents =  events[pd.to_numeric(events['person'], errors='coerce').notnull()] # removes drivers
agents.head()
agents.vehicle.unique()

array([nan, '2451', '1083', ..., 'veh_2194_bus', 'veh_12613_bus',
       'veh_23236_tram'], dtype=object)

In [13]:
"""
people = [
        {id: 1,
        events: [
                {event_id :,
                 time: 
                 type:
                 link:
                 vehicle_id:
                 delay:
                 coords: [x,y]
                 destination:  Act)
                 }
        ]}
]
"""

def save_agents(persons, json_file):
    people = []
    for person in persons:
        agent = pd.DataFrame(person[1])
        agent_id = person[0]
        agent = agent.sort_values("time")
        #print("Person Id:", agent_id)
        events = []
        for id, row in agent.iterrows():
            event = {}
            event["event_id"] = row[0]
            event["time"] = row["time"]
            event["type"] = row["type"]
            event["link"] = row["link"]
            event["vehicle_id"] = row["vehicle"]
            event["delay"] = row["delay"]
            event["actType"] = row["actType"]
            event["legMode"] = row["legMode"]
            event["coords_x"] = row["x"]
            event["coords_y"] = row["y"]
            events.append(event)

        people.append({
            "id": agent_id,
            "events": events
        })

    with open(json_file, 'w') as f:
        json.dump(people, f)

In [14]:
save_agents(agents.groupby("person"), json_file = "./output/events/agents.json")

## Loading vehicle events to separate files

- cars
- subway
- buses
- trams
- trains
- ferry
- funicular


In [80]:
def save_vehicle(vehicle_events, json_file, vehicle_type = ""):
    vehicles = pd.DataFrame()
    for agent in vehicle_events:
        vehicle = pd.DataFrame(agent[1])
        vehicle = vehicle.sort_values("time")
        events = []
        if(vehicle_type == "car"):
            vehicle_id = int(vehicle.vehicle.unique()[0])
        else:
            vehicle_id = vehicle.vehicle.unique()[0]

        #parse events
        for id, row in vehicle.iterrows():
            event = {}
            event["event_id"] = row[0]
            event["time"] = row["time"]
            event["type"] = row["type"]
            event["link"] = row["link"]
            event["person_id"] = row["person"]
            event["delay"] = row["delay"]
            event["facility"] = row['facility']
            #if facility, parse out link
            #print(type(row['facility']))
            if isinstance(row['facility'],str): #and not(np.isnan(row['facility'])):
                #print("filling link", row['facility'],row['facility'].split(":")[-1])
                event['link'] = row['facility'].split(":")[-1]
                #return
            event["networkMode"] = row['networkMode']
            event["relativePosition"] = row['relativePosition']
            event["actType"] = row["actType"]
            event["legMode"] = row["legMode"]
            event["coords_x"] = row["x"]
            event["coords_y"] = row["y"]

            if(vehicle_type != "car"):
                if(event["type"] == "TransitDriverStarts"):
                    #print("Driver starts")
                    event["transitLine"] = row['transitLineId']
                    event["transitRoute"] = row['transitRouteId'] ## add to output
                    #if( not np.isnan(row['transitRouteId'])):
                    #    print(event["transitRoute"], type(event["transitRoute"]))
                    #if( not (row['transitLineId'] == 'nan')):
                    #    print(event["transitLine"], type(event["transitLine"]))
                event["departure"] = row['departureId']
                event["atStop"] = row["atStop"]
                event["destinationStop"] = row["destinationStop"]

            events.append(event)

        new_vehicle = pd.DataFrame(columns=['id','events'])
        new_vehicle.loc[0,'id'] = vehicle_id
        new_vehicle.loc[0,'events']  =  events
        vehicles = vehicles.append(new_vehicle)

    vehicles.reset_index(drop=True, inplace=True)
    vehicles.to_json(json_file+"/"+vehicle_type+".json", lines=True, orient='records')  
    del vehicles 


In [81]:
vehicle_types = ["car","funicular","ferry","subway","rail","tram","bus"]

def filter_vehicle_events(events, vehicle_type, json_path="./output/events"):
    from datetime import datetime
    print("Started filtering:",vehicle_type, "at", datetime.now())
    vehicles = pd.DataFrame()
    if vehicle_type == 'car':
        vehicles = events[pd.to_numeric(events['vehicle'], errors='coerce').notnull()]
    else:
        vehicles = events.loc[events['vehicle'].str.contains(vehicle_type, case=False)]
        #events['vehicleId'].notnull() and 
        driver_events = events[events['vehicleId'].notnull() & events['vehicleId'].str.contains(vehicle_type, case=False)]
        driver_events['vehicle'] = driver_events['vehicleId']
        
        #display(driver_events.head())
        #return
        vehicles = vehicles.append(driver_events)

    save_vehicle(vehicles.groupby("vehicle"), json_file = json_path, vehicle_type=vehicle_type)
    return print("Saved vehicle type:",vehicle_type, "to", json_path, "at", datetime.now())
        

In [84]:
for veh_type in ['subway','tram','ferry','bus',]:
        events.vehicle = events.vehicle.astype("string")
        filter_vehicle_events(events, veh_type)

Started filtering: subway at 2021-11-09 13:16:53.263225


C:\Users\Anna\AppData\Local\Temp/ipykernel_14820/1838754327.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  driver_events['vehicle'] = driver_events['vehicleId']


Saved vehicle type: subway to ./output/events at 2021-11-09 13:18:44.524946
Started filtering: tram at 2021-11-09 13:18:46.308175
Saved vehicle type: tram to ./output/events at 2021-11-09 13:26:18.138561
Started filtering: ferry at 2021-11-09 13:26:18.775897
Saved vehicle type: ferry to ./output/events at 2021-11-09 13:26:30.693072
Started filtering: bus at 2021-11-09 13:26:31.239961
Saved vehicle type: bus to ./output/events at 2021-11-09 13:43:16.567825
